In [ ]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, LSTM, Bidirectional, Input
from tensorflow.keras import layers

import numpy as np
import pickle

pickle_in = open("X_train_encoded_full_2D.pickle","rb")
X_train = pickle.load(pickle_in)

pickle_in = open("y_train_full.pickle","rb")
y_train = pickle.load(pickle_in)


pickle_in = open("X_test_encoded_full_2D.pickle","rb")
X_test = pickle.load(pickle_in)

pickle_in = open("y_test_full.pickle","rb")
y_test = pickle.load(pickle_in)

# layer = tf.keras.layers.LayerNormalization(axis=[2])
# output = layer(X_test)

In [ ]:
image_size = (14, 11)
batch_size = 32

In [ ]:
from tensorflow import keras 

def dct_2d(
        feature_map,
        norm=None # can also be 'ortho'
):
    X1 = tf.signal.dct(feature_map, type=2, norm=norm)
    X1_t = tf.transpose(X1, perm=[0, 1, 3, 2])
    X2 = tf.signal.dct(X1_t, type=2, norm=norm)
    X2_t = tf.transpose(X2, perm=[0, 1, 3, 2])
    return X2_t

def make_model(input_shape, num_classes):
    inputs = layers.Input(shape=input_shape)
    # 2D DCT block
    x=layers.Lambda(dct_2d)(inputs)
    x= layers.Permute((2, 3,1))(x)
    
    # Entry block
    #x = layers.experimental.preprocessing.Rescaling(1.0 / 255)(x)
    x = layers.LayerNormalization(axis=1)(x)
    x = layers.Conv2D(32, 3, strides=2, padding="same")(x)
    x = layers.BatchNormalization()(x)
    x = layers.Activation("relu")(x)

    x = layers.Conv2D(64, 3, padding="same")(x)
    x = layers.BatchNormalization()(x)
    x = layers.Activation("relu")(x)

    previous_block_activation = x  # Set aside residual

    for size in [128, 256, 512, 728]:
        x = layers.Activation("relu")(x)
        x = layers.SeparableConv2D(size, 3, padding="same")(x)
        x = layers.BatchNormalization()(x)

        x = layers.Activation("relu")(x)
        x = layers.SeparableConv2D(size, 3, padding="same")(x)
        x = layers.BatchNormalization()(x)

        x = layers.MaxPooling2D(3, strides=2, padding="same")(x)

        # Project residual
        residual = layers.Conv2D(size, 1, strides=2, padding="same")(
            previous_block_activation
        )
        x = layers.add([x, residual])  # Add back residual
        previous_block_activation = x  # Set aside next residual

    x = layers.SeparableConv2D(1024, 3, padding="same")(x)
    x = layers.BatchNormalization()(x)
    x = layers.Activation("relu")(x)

    x = layers.GlobalAveragePooling2D()(x)
    if num_classes == 2:
        activation = "sigmoid"
        units = 1
    else:
        activation = "softmax"
        units = num_classes

    x = layers.Dropout(0.5)(x)
    outputs = layers.Dense(units, activation=activation)(x)
    return keras.Model(inputs, outputs)


model = make_model(input_shape=(160,)+ image_size, num_classes=8)
keras.utils.plot_model(model, show_shapes=True)

In [ ]:
from tensorflow.keras.callbacks import ModelCheckpoint

epochs = 50

"""
callbacks = [
    keras.callbacks.ModelCheckpoint("save_at_{epoch}.h5"),
]
"""

# checkpoint
filepath="semantic_2D_DCT_v1.weights.best.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='val_accuracy', verbose=1, save_best_only=True, mode='max')
callbacks_list = [checkpoint]

model.compile(
    optimizer=keras.optimizers.Adam(1e-3),
    loss="sparse_categorical_crossentropy",
    metrics=["accuracy"],
)




In [ ]:
# serialize model to JSON
model_json = model.to_json()
with open("semantic_2D_DCT_v1.json", "w") as json_file:
    json_file.write(model_json)
# serialize weights to HDF5
model.save_weights("semantic_2D_DCT_v1.h5")
print("Saved model to disk")

In [ ]:
model.fit(
    X_train,np.array(y_train), epochs=epochs, callbacks=callbacks_list, validation_data=(X_test, np.array(y_test)),
)

